In [4]:
%pip install nltk
import nltk
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import re
df = pd.read_csv("test.csv")

suf = []
words = []

with open("suf.txt", "r", encoding="utf-8-sig") as s:
    suf = s.read().split("\n")
with open("dict.txt", "r", encoding="utf-8-sig") as s:
    words = s.read().split("\n")

tokenizer = RegexpTokenizer(r'[\u0900-\u097F]+')  # Matches Devanagari script words
def stem(text, words_list=suf):
    if text in suf:
        return text

    w = []
    for t in text.split(" "):
        if t not in words:
            pattern = r'(\s*)(' + '|'.join(map(re.escape, words_list)) + r')$'
            modified_text = re.sub(pattern, r' \2', t)
            if len(modified_text.split(" "))>1:
                modified_text = stem(modified_text.split(" ")[0]) + " " + modified_text.split(" ")[1]
            w.append(modified_text)
        else:
            w.append(t)
        
    modified_text = " ".join(w)
    
    return modified_text

text = "यो नक्कली समाचार हो।"
tokens = tokenizer.tokenize(text)
print(tokens) 
df = df.fillna("")
df["text"] = df["text"].apply(stem)

Note: you may need to restart the kernel to use updated packages.
['यो', 'नक्कली', 'समाचार', 'हो।']


In [38]:
df

,Unnamed: 0,text,class,sequences,padded_sequences
0,0,नतिजा हेर्न को लागि लगइन गर्नुहोस् ।,1,"[1137, 1070, 2, 16, 7842, 4719, 11]","[1137, 1070, 2, 16, 7842, 4719, 11, 0, 0, 0, 0..."
1,1,तपाई को प्रतिक्रिया समीक्षा मा भए को ले प्रकाश...,1,"[1667, 2, 1184, 1307, 3, 9, 2, 5, 1578, 9, 2, ...","[1667, 2, 1184, 1307, 3, 9, 2, 5, 1578, 9, 2, ..."
2,2,तपाई को प्रतिक्रिया समीक्षा मा भए को ले प्रकाश...,1,"[1667, 2, 1184, 1307, 3, 9, 2, 5, 1578, 9, 2, ...","[1667, 2, 1184, 1307, 3, 9, 2, 5, 1578, 9, 2, ..."
3,3,देश मा पश्चिमी वायु को सामान्य प्रभाव रहेको ब...,1,"[55, 3, 1722, 1185, 2, 623, 600, 18, 602, 1997...","[55, 3, 1722, 1185, 2, 623, 600, 18, 602, 1997..."
4,4,"‘आज त आउँछ होला यो विषय, कसै ले त उठाउँछ होला ...",1,"[165, 133, 2615, 645, 26, 96, 405, 5, 133, 784...","[165, 133, 2615, 645, 26, 96, 405, 5, 133, 784..."
...,...,...,...,...,...
2997,2997,उहाँ ले हल भित्र स्पष्ट रुप ले बोली बुझिने को ...,0,"[742, 5, 3558, 238, 431, 4208, 5, 2144, 1, 2, ...","[742, 5, 3558, 238, 431, 4208, 5, 2144, 1, 2, ..."
2998,2998,कुनै पनि विषय सरकार वा रास्ट्रपति विरुद्ध अफवा...,0,"[150, 12, 96, 25, 192, 3009, 280, 2632, 2584, ...","[150, 12, 96, 25, 192, 3009, 280, 2632, 2584, ..."
2999,2999,यति धेरै नगरे को भए हुने हाे राष्टपति जस्तो मा...,0,"[1039, 157, 1175, 2, 9, 33, 1282, 1, 326, 627,...","[1039, 157, 1175, 2, 9, 33, 1282, 1, 326, 627,..."
3000,3000,"रमेश प्रसाईं काठमान्डौ आइ सके को छ , यो दिल नि...",0,"[1045, 556, 1, 846, 1000, 2, 21, 26, 5161, 1, ...","[1045, 556, 1, 846, 1000, 2, 21, 26, 5161, 1, ..."


In [60]:

s = re.sub(r"[$&+।,:;=?@#|'<>.^*()%!-]", "", "hello । man")
s

'hello  man'

In [65]:
import re
from nltk.tokenize import RegexpTokenizer
from collections import Counter

def clean_text(text):
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip() 
    puncts = r"$&+,:;=?@#|'<>.^*()%!-।"
    if text in list(puncts):
        return text
    else:
        text = re.sub(r"[$&+।।,:;=?@#|'<>.^*()%!-]","", text)

    return text
tokenizer = RegexpTokenizer(r'[\u0900-\u097F]+') 

def build_vocab(tokenized_texts, max_vocab_size=10000):
    all_tokens = [token for tokens in tokenized_texts for token in tokens]
    word_counts = Counter(all_tokens)
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(word_counts.most_common(max_vocab_size))}
    vocab["<PAD>"] = 0  
    vocab["<UNK>"] = 1  
    return vocab

def text_to_sequence(tokens, vocab):
    return [vocab.get(token, vocab["<UNK>"]) for token in tokens]

def preprocess_data(df, vocab):
    tokenized_texts = []
    for text in df['text']:
        cleaned_text = clean_text(text)
        tokens = tokenizer.tokenize(cleaned_text)
        tokenized_texts.append(tokens)
    
    sequences = [text_to_sequence(tokens, vocab) for tokens in tokenized_texts]
    
    return sequences

tokenized_texts = [tokenizer.tokenize(clean_text(text)) for text in df['text']]
vocab = build_vocab(tokenized_texts)

sequences = preprocess_data(df, vocab)
df['sequences'] = sequences


In [66]:
df

,Unnamed: 0,text,class,sequences,padded_sequences
0,0,नतिजा हेर्न को लागि लगइन गर्नुहोस् ।,1,"[1128, 1066, 2, 16, 7766, 4330]","[1137, 1070, 2, 16, 7842, 4719, 11, 0, 0, 0, 0..."
1,1,तपाई को प्रतिक्रिया समीक्षा मा भए को ले प्रकाश...,1,"[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, 69]","[1667, 2, 1184, 1307, 3, 9, 2, 5, 1578, 9, 2, ..."
2,2,तपाई को प्रतिक्रिया समीक्षा मा भए को ले प्रकाश...,1,"[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, 69]","[1667, 2, 1184, 1307, 3, 9, 2, 5, 1578, 9, 2, ..."
3,3,देश मा पश्चिमी वायु को सामान्य प्रभाव रहेको ब...,1,"[54, 3, 1709, 1178, 2, 620, 595, 18, 600, 1986...","[55, 3, 1722, 1185, 2, 623, 600, 18, 602, 1997..."
4,4,"‘आज त आउँछ होला यो विषय, कसै ले त उठाउँछ होला ...",1,"[160, 128, 2243, 596, 26, 95, 397, 5, 128, 776...","[165, 133, 2615, 645, 26, 96, 405, 5, 133, 784..."
...,...,...,...,...,...
2997,2997,उहाँ ले हल भित्र स्पष्ट रुप ले बोली बुझिने को ...,0,"[737, 5, 3543, 234, 420, 4186, 5, 2134, 1, 2, ...","[742, 5, 3558, 238, 431, 4208, 5, 2144, 1, 2, ..."
2998,2998,कुनै पनि विषय सरकार वा रास्ट्रपति विरुद्ध अफवा...,0,"[147, 11, 95, 25, 188, 2987, 274, 2616, 2575, ...","[150, 12, 96, 25, 192, 3009, 280, 2632, 2584, ..."
2999,2999,यति धेरै नगरे को भए हुने हाे राष्टपति जस्तो मा...,0,"[1032, 153, 1167, 2, 9, 33, 1271, 1, 319, 618,...","[1039, 157, 1175, 2, 9, 33, 1282, 1, 326, 627,..."
3000,3000,"रमेश प्रसाईं काठमान्डौ आइ सके को छ , यो दिल नि...",0,"[1039, 549, 1, 836, 966, 2, 7, 26, 5109, 1, 10...","[1045, 556, 1, 846, 1000, 2, 21, 26, 5161, 1, ..."


In [67]:
%pip install torch
import torch

def pad_sequences(sequences, max_length):
    padded = []
    for seq in sequences:
        if len(seq) > max_length:
            padded.append(seq[:max_length])
        else:
            padded.append(seq + [0] * (max_length - len(seq)))  
    return padded

max_length = 100
padded_sequences = pad_sequences(sequences, max_length)

df['padded_sequences'] = padded_sequences

Note: you may need to restart the kernel to use updated packages.


In [68]:
df

,Unnamed: 0,text,class,sequences,padded_sequences
0,0,नतिजा हेर्न को लागि लगइन गर्नुहोस् ।,1,"[1128, 1066, 2, 16, 7766, 4330]","[1128, 1066, 2, 16, 7766, 4330, 0, 0, 0, 0, 0,..."
1,1,तपाई को प्रतिक्रिया समीक्षा मा भए को ले प्रकाश...,1,"[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, 69]","[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, ..."
2,2,तपाई को प्रतिक्रिया समीक्षा मा भए को ले प्रकाश...,1,"[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, 69]","[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, ..."
3,3,देश मा पश्चिमी वायु को सामान्य प्रभाव रहेको ब...,1,"[54, 3, 1709, 1178, 2, 620, 595, 18, 600, 1986...","[54, 3, 1709, 1178, 2, 620, 595, 18, 600, 1986..."
4,4,"‘आज त आउँछ होला यो विषय, कसै ले त उठाउँछ होला ...",1,"[160, 128, 2243, 596, 26, 95, 397, 5, 128, 776...","[160, 128, 2243, 596, 26, 95, 397, 5, 128, 776..."
...,...,...,...,...,...
2997,2997,उहाँ ले हल भित्र स्पष्ट रुप ले बोली बुझिने को ...,0,"[737, 5, 3543, 234, 420, 4186, 5, 2134, 1, 2, ...","[737, 5, 3543, 234, 420, 4186, 5, 2134, 1, 2, ..."
2998,2998,कुनै पनि विषय सरकार वा रास्ट्रपति विरुद्ध अफवा...,0,"[147, 11, 95, 25, 188, 2987, 274, 2616, 2575, ...","[147, 11, 95, 25, 188, 2987, 274, 2616, 2575, ..."
2999,2999,यति धेरै नगरे को भए हुने हाे राष्टपति जस्तो मा...,0,"[1032, 153, 1167, 2, 9, 33, 1271, 1, 319, 618,...","[1032, 153, 1167, 2, 9, 33, 1271, 1, 319, 618,..."
3000,3000,"रमेश प्रसाईं काठमान्डौ आइ सके को छ , यो दिल नि...",0,"[1039, 549, 1, 836, 966, 2, 7, 26, 5109, 1, 10...","[1039, 549, 1, 836, 966, 2, 7, 26, 5109, 1, 10..."


In [69]:
from torch.utils.data import DataLoader, TensorDataset

sequences_tensor = torch.tensor(padded_sequences)
labels_tensor = torch.tensor(df['class'].values)

dataset = TensorDataset(sequences_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for batch in dataloader:
    inputs, labels = batch
    print(inputs, labels)

tensor([[3146, 7054,  327,    3, 7302,    3,  971,   37, 3992,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   4,    5,    4,    5,  858, 9648, 7154, 2600,   92,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 

In [70]:
import torch.nn as nn
import torch.optim as optim

class FakeNewsLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(FakeNewsLSTM, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers=n_layers, 
                            bidirectional=bidirectional, 
                            dropout=dropout, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        lstm_out, (hidden, cell) = self.lstm(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)) if self.lstm.bidirectional else hidden[-1,:,:]
        output = self.fc(hidden)
        return self.sigmoid(output)

embedding_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.5
model = FakeNewsLSTM(len(vocab), embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch in dataloader:
        text, labels = batch
        optimizer.zero_grad()
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, labels.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {epoch_loss / len(dataloader)}')

Epoch 1, Loss: 0.22488641246342234
Epoch 2, Loss: 0.07552541763140327
Epoch 3, Loss: 0.04541671738745857


In [76]:
sss = input()
sss = pd.DataFrame([sss], columns=["text"])
sss["text"] = sss["text"].apply(stem)
ss = preprocess_data(sss, vocab)
sss['sequences'] = ss
max_length = 100
ps = pad_sequences(ss, max_length)
ip = torch.tensor(ps)
op = model(ip)
op

 गोर्खा मिडिया नेटवर्कका तत्कालीन प्रबन्ध निर्देशक रवि लामिछाने चढ्ने गाडीसमेत स्वर्णलक्ष्मी सहकारीको बचत अपचलनबाट खरिद गरिएको पुष्टि भएको छ । सवा दुई करोडको ‘टोयोटा फर्च्युनर’ गाडी किन्नेसहित विभिन्न प्रयोजनका लागि स्वर्णलक्ष्मीबाट निकालिएको ऋण भुक्तान गर्न लामिछाने आफैंले काटेका चेक पनि भेटिएका छन् ।


tensor([[0.9992]], grad_fn=<SigmoidBackward0>)

In [78]:
import pickle
with open('PT.pkl', 'wb') as f:
    pickle.dump(model, f)

In [79]:
df.to_csv("test2.csv")

In [105]:
df["class"].value_counts()[0]>df["class"].value_counts()[1]

np.False_

In [108]:
df = pd.read_csv("test2.csv")

In [110]:
df["class"].value_counts()

,Unnamed: 0.1,Unnamed: 0,text,class,sequences,padded_sequences
0,0,0,नतिजा हेर्न को लागि लगइन गर्नुहोस् ।,1,"[1128, 1066, 2, 16, 7766, 4330]","[1128, 1066, 2, 16, 7766, 4330, 0, 0, 0, 0, 0,..."
1,1,1,तपाई को प्रतिक्रिया समीक्षा मा भए को ले प्रकाश...,1,"[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, 69]","[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, ..."
2,2,2,तपाई को प्रतिक्रिया समीक्षा मा भए को ले प्रकाश...,1,"[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, 69]","[1658, 2, 1177, 1296, 3, 9, 2, 5, 1572, 9, 2, ..."
3,3,3,देश मा पश्चिमी वायु को सामान्य प्रभाव रहेको ब...,1,"[54, 3, 1709, 1178, 2, 620, 595, 18, 600, 1986...","[54, 3, 1709, 1178, 2, 620, 595, 18, 600, 1986..."
4,4,4,"‘आज त आउँछ होला यो विषय, कसै ले त उठाउँछ होला ...",1,"[160, 128, 2243, 596, 26, 95, 397, 5, 128, 776...","[160, 128, 2243, 596, 26, 95, 397, 5, 128, 776..."
...,...,...,...,...,...,...
2997,2997,2997,उहाँ ले हल भित्र स्पष्ट रुप ले बोली बुझिने को ...,0,"[737, 5, 3543, 234, 420, 4186, 5, 2134, 1, 2, ...","[737, 5, 3543, 234, 420, 4186, 5, 2134, 1, 2, ..."
2998,2998,2998,कुनै पनि विषय सरकार वा रास्ट्रपति विरुद्ध अफवा...,0,"[147, 11, 95, 25, 188, 2987, 274, 2616, 2575, ...","[147, 11, 95, 25, 188, 2987, 274, 2616, 2575, ..."
2999,2999,2999,यति धेरै नगरे को भए हुने हाे राष्टपति जस्तो मा...,0,"[1032, 153, 1167, 2, 9, 33, 1271, 1, 319, 618,...","[1032, 153, 1167, 2, 9, 33, 1271, 1, 319, 618,..."
3000,3000,3000,"रमेश प्रसाईं काठमान्डौ आइ सके को छ , यो दिल नि...",0,"[1039, 549, 1, 836, 966, 2, 7, 26, 5109, 1, 10...","[1039, 549, 1, 836, 966, 2, 7, 26, 5109, 1, 10..."


In [112]:
with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)